In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

print(os.getcwd())
sys.path.append(str(Path('../src').resolve()))

/media/dicarlod/SSD_2/diego/Code/INCANT/python/notebooks


In [48]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

import scipy.interpolate as interpolate
from matplotlib.colors import LogNorm

from tqdm import tqdm

from implicits import SIREN, MFN, continuous_diff

In [6]:
# Load Data
path_to_csv0 = Path("../../data/clippeddata/clippedMarco0.txt")
path_to_csv1 = Path("../../data/clippeddata/clippedMarco1.txt")

data_t0 = pd.read_csv(path_to_csv0).to_numpy()[:,:3]
data_t0 = np.concatenate([data_t0, np.zeros((data_t0.shape[0],1))], axis=-1)
print('Data t0:', data_t0.shape)
data_t1 = pd.read_csv(path_to_csv1).to_numpy()[:,:3]
data_t1 = np.concatenate([data_t1, np.ones((data_t1.shape[0],1))], axis=-1)
print('Data t1:', data_t1.shape)

data = np.concatenate([data_t0, data_t1], axis=0)
col_names = ["x","y","z","t"]

print("min", np.min(data, axis=0))
print("mean", np.mean(data, axis=0))
print("max", np.max(data, axis=0))

Data t0: (24253, 4)
Data t1: (26658, 4)
min [  6.375 216.5   248.309   0.   ]
mean [ 36.41119797 238.84924672 261.26950427   0.52361965]
max [ 64.5   261.5   275.804   1.   ]


In [ ]:
# Create dataloader
class MarcoDataset(torch.utils.data.Dataset):
    def __init__(self, data, col_names, device):
        self.col_names = col_names
        self.data = data
        n_obs = self.data.shape[0]
        
        self.coords = torch.Tensor(self.data[:,[0,1,3]]).to(device)
        self.target = torch.Tensor(self.data[:,2][:,None]).to(device)
        
        # normalization between [-1,1] for SIREN
        self.coords = self.coords / torch.abs(torch.max(self.coords, axis=0, keepdim=True)[0])
        for d in range(self.coords.shape[1]):
            assert torch.max(self.coords[:,d][0]) <  1
            assert torch.max(self.coords[:,d][0]) > -1
        
        print(self.coords.shape)
        print(self.target.shape)
        
        
    def __len__(self):
        return self.coords.shape[0]
    
    def __getitem__(self, idx):
        coords = self.coords[idx,:]
        target = self.target[idx,:]
        return (coords, target)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
dset = MarcoDataset(data, col_names, device)

n_obs = len(dset)
print(n_obs)

n_test = int((10 / 100) * n_obs)
n_val  = int((10 / 100) * n_obs)
n_train = n_obs - n_val - n_test

dset_train, dset_val, dset_test = torch.utils.data.random_split(dset, [n_train, n_val, n_test])
dload_train = torch.utils.data.DataLoader(dset_train, shuffle=True, batch_size=1024)
dload_val = torch.utils.data.DataLoader(dset_val, shuffle=True, batch_size=n_val)
dload_test = torch.utils.data.DataLoader(dset, shuffle=False, batch_size=n_obs)

In [ ]:
lr = 1e-3
epochs = 300
val_interval = 25
loss_name = 'MSE'

losses = {
    'MSE' : torch.nn.MSELoss(),
    'MAE' : torch.nn.L1Loss(),
}

class INCANT(SIREN):
    def __init__(self, in_dim, out_dim, hidden_num, hidden_dim, feature_scales):
        super(INCANT, self).__init__(in_dim, out_dim, hidden_num, hidden_dim, feature_scales)
    
    # extend SIREN forward
    def forward(self, xin):
        return 100 * torch.tanh(super(INCANT, self).forward(xin)) + 260

model = INCANT(
    in_dim = 3,
    out_dim = 1,
    hidden_num = 3,
    hidden_dim = 512,
    feature_scales = [30., 30., 10]
)
optim = torch.optim.Adam(lr=lr, params=model.parameters())
loss_fun = losses[loss_name]
        
loss_track = []
metrics_track = []

model = model.to(device)
model.train()


for step in tqdm(range(epochs)):

    for batch_idx, (coords, reference) in enumerate(dload_train):

        # model with some prior knowledge
        estimated = model(coords)
        loss = loss_fun(estimated, reference)
        
        if not step % val_interval and batch_idx == 0:

            print("Step %d, Total loss %0.6f" % (step, loss))

            model.eval()
            coords, reference = next(iter(dload_val))
            estimated = model(coords)

            # metrics = compute_metrics(estimated, reference)
            metrics = {
                'mse' : (torch.abs(reference - estimated)**2).cpu().detach().numpy()
            }
            model.train()

        optim.zero_grad()
        loss.backward()
        optim.step()

        metrics_track.append(metrics)
        loss_track.append(loss.item())

results_dict = {
    'conf' : None,
    'loss' : np.array(loss_track),
    'metrics' : metrics_track
}
# with open(outdir / Path('res_dict.pkl'), 'wb') as f:
#     pickle.dump(results_dict, f)

In [ ]:
plt.plot(np.log(loss_track))
plt.show()